# 트랜스포머 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.6 MB/s eta 0:00:00


# koGPT 로드(허깅 페이스로부터 로드)

- 레퍼지토리 이름 : skt/kogpt2-base-v2


- 스폐셜 토큰
    - 토크나이저가 텍스트를 인코딩할 때
        - 문장의 시작, 문장의 끝, 문장과 문장사이 구분, 패딩 처리 등에 기호로 사용되는 토큰
        - <이름> or </이름> 활용

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
repo = 'skt/kogpt2-base-v2'                  
# 사전에 토크나이저에 사용된 스페셜 토근외에 커스텀으로 토큰을 변경 지정할 수 있다
# 지정한 스페셜 토큰이 기존값과 동일하면 기존 번호를 계승함, 다르면 사전에 추가된다 51201번부터 추가됨
tokenizer = AutoTokenizer.from_pretrained(repo, bos_token='</s>', eos_token='</s>', pad_token='<pad>' )
model = TFGPT2LMHeadModel.from_pretrained( repo, from_pt=True ) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

# 커스텀 데이터 준비



## 스폐셜 토큰
- 준비된 스페셜 토큰 샘플 확인
- koGPT2는 문장과 문장 사이 전후에 스페셜 토큰을 넣어서 메타 정보를 구성한다

In [ ]:
# 문장시작, 문장끝, 패딩
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id

(1, 1, 3)

In [ ]:
tokenizer.decode(1), tokenizer.decode(2), tokenizer.decode(3), tokenizer.decode(4)
# <sys> 문장과 문장 사이 구분자
# </s><usr>질문<sys>답변</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>...<pad>

('</s>', '<usr>', '<pad>', '<sys>')

## 데이터 획득

In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:
data_url = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv'
urllib.request.urlretrieve(data_url, filename='ChatbotData.csv')

('ChatbotData.csv', <http.client.HTTPMessage at 0x7f488dea7d60>)

In [ ]:
train_raw_data = pd.read_csv('ChatbotData.csv')
train_raw_data.head(1)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0


In [ ]:
for q, a in zip(train_raw_data.Q, train_raw_data.A):
    print(q, a)
    break

12시 땡! 하루가 또 가네요.


In [ ]:
def generator():
    # 데이터를 하나씩 뽑아서 아래 작업을 끝까지 반복 -> for
    for q, a in zip(train_raw_data.Q, train_raw_data.A):
        # 스페셜 토큰을 붙여서 인코딩한다
        # 하나씩 리턴해주면서 공급한다 (반복문 안에서 값을 계속 돌려준다, 반복문은 유지됨-> yield)
        bos = [ tokenizer.bos_token_id ]
        sentence = tokenizer.encode( f'{tokenizer.decode(2)}{q}{tokenizer.decode(4)}{a}' )
        eos = [ tokenizer.eos_token_id ] 
        # yield [ </s>, <usr>, 질문, <sys>, 답변,  </s> ]
        yield bos + sentence + eos  # generator (필요할 때만 데이터를 생성, 메모리에 1개씩만 올려줌, 대용량 데이터에 효율적)
        # print(bos + sentence + eos)
        # return 

# 데이터 공급자
dataset = tf.data.Dataset.from_generator(generator, output_types=tf.int32, )
# generator()

# dataset을 사용할 때 필요한 양만큼 그때 데이터가 생성됨

In [ ]:
for value in dataset:
    print(value, value.shape, type(value))
    break

tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1], shape=(15,), dtype=int32) (15,) <class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
# 배치사이즈 지정 => 1회 학습시 32개 데이터를 공급
batch_size = 32
padded_dataset = dataset.padded_batch(batch_size=batch_size,
                     padded_shapes=(None, ),
                     padding_values=tokenizer.pad_token_id)
for value in padded_dataset:
    print(value[0], value.shape, type(value))
    break

tf.Tensor(1, shape=(), dtype=int32) (32, 30) <class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
# 디코딩을 통해 문장 확인
print(tokenizer.decode( value[0] ))
print(tokenizer.decode( value[1] ))
print(tokenizer.decode( value[2] ))

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
</s><usr> 1지망 학교 떨어졌어<sys> 위로해 드립니다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
</s><usr> 3박4일 놀러가고 싶다<sys> 여행은 언제나 좋죠.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


# 학습에 필요한 도구 정리(컴파일)

- 최적화 도구 지정, 1세대에 필요한 총 학습 횟수 계산함
    - 직접 훈련을 진행하겠다
    - keras로 하지 않고, tensorflow로 진행한다

In [ ]:
# learning_rate :학습율
# epsilon :최적화 작업시 수치 안정화를 위한 상수
# e를 이용하여 설정값을 부여
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-9)

In [ ]:
train_step = len(train_raw_data) // batch_size + 1
train_step

370

# 커스텀 데이터를 이용하여 학습 진행

- 이 코드는 tensorflow low 레벨로 직접 학습을 수행한 코드
- 차후에는 fit() 함수를 통해서 진행

In [ ]:
EPOCHS = 20
adam = tf.keras.optimizers.Adam(learning_rate = 3e-5, epsilon=1e-08)
steps = len(train_raw_data) // batch_size + 1

In [ ]:
import time
import datetime

In [ ]:
start = time.time()
# dataset을 batch_size으로 나눈 batch에서 loss를 계산한 후
# tf.GradientTape으로 gradient을 계산하고
# 계산한 gradient을 통해 adam(model)을 업데이트합니다.

for epoch in range(EPOCHS):
    train_loss = 0

    try:
        for batch in tqdm.notebook.tqdm(padded_dataset, total = steps):
            try:
                with tf.GradientTape() as tape:
                    result = model(batch, labels = batch)
                    loss = result[0]
                    batch_loss = tf.reduce_mean(loss, -1)
      
                grads = tape.gradient(batch_loss, model.trainable_variables)
                adam.apply_gradients(zip(grads, model.trainable_variables))
                train_loss += batch_loss / steps
                
            except:
                pass
            
    except:
        pass

end = time.time()
print(str(datetime.timedelta(seconds=(end - start))))

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

...

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

  0%|          | 0/370 [00:00<?, ?it/s]

1:03:54.207141


# 저장하기

- 토크나이저, 모델 저장(덤프)
- 디렉토리를 실제 생성 후 지정

In [ ]:
# model.save_pretrained('저장할 경로 지정')
model.save_pretrained('chatbot_model')
tokenizer.save_pretrained('chatbot_model')

('/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/tokenizer_config.json',
 '/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/special_tokens_map.json',
 '/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/vocab.json',
 '/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/merges.txt',
 '/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/added_tokens.json',
 '/content/drive/MyDrive/SK 쉴더스/모듈프젝3/chatbot_model/tokenizer.json')